In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
data = tfds.load('yelp_polarity_reviews', split='train', shuffle_files=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/yelp_polarity_reviews/plain_text/0.1.0.incomplete0IAOZ3/yelp_polarity_reviews-train.tfrecord


  0%|          | 0/560000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/yelp_polarity_reviews/plain_text/0.1.0.incomplete0IAOZ3/yelp_polarity_reviews-test.tfrecord


  0%|          | 0/38000 [00:00<?, ? examples/s]

Dataset yelp_polarity_reviews downloaded and prepared to /root/tensorflow_datasets/yelp_polarity_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [ ]:
reviews = []
polarity = []

for i in data.take(20000):
    reviews.append((i['text'].numpy().decode("utf-8")))
    polarity.append(int(i['label']))

In [ ]:
reviews[3]

"It was bad. My nails are ruined. I asked for purple and sparkles and they gave me nothing near that. I got too many sparkles. They didn't even ask me if I liked it. My friend who went with me got 2 coats and a top. I got not even a full 1st coat it was the worst. I am going somewhere else tomorrow. I had the worse time ever. Looks like Rainbow nails will not be seeing  me there any time soon!!!!!!!!!!!!!"

In [ ]:
def clean_text(review):
    cleaned = review.replace("\\n", " ")
    cleaned = cleaned.replace("\'", "'")
    cleaned = cleaned.replace("\\r", " ")
    cleaned = cleaned.replace("\\""", " ")
    return cleaned

In [ ]:
reviews = [clean_text(review) for review in reviews]

In [ ]:
reviews[3]

"It was bad. My nails are ruined. I asked for purple and sparkles and they gave me nothing near that. I got too many sparkles. They didn't even ask me if I liked it. My friend who went with me got 2 coats and a top. I got not even a full 1st coat it was the worst. I am going somewhere else tomorrow. I had the worse time ever. Looks like Rainbow nails will not be seeing  me there any time soon!!!!!!!!!!!!!"

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
max_length = 200

tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(reviews)

In [ ]:
sequences = tokenizer.texts_to_sequences(reviews)
padded_sequences = pad_sequences(sequences, max_length, padding = 'post')

In [ ]:
padded_sequences

array([[ 37,  20, 136, ...,   0,   0,   0],
       [  5,  47,  30, ...,   0,   0,   0],
       [100,  46, 374, ...,   0,   0,   0],
       ...,
       [ 42, 317,  23, ...,   0,   0,   0],
       [  3, 214,  10, ...,   0,   0,   0],
       [  6, 181,  17, ...,  49,  15,  30]], dtype=int32)

In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(padded_sequences, polarity)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y)

In [ ]:
len(train_x), len(train_y)


(11250, 11250)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten

model = Sequential()
model.add(Embedding(vocab_size, 64))
model.add(Bidirectional(LSTM(20, return_sequences = True)))
model.add(Bidirectional(LSTM(20)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 64)          640000    
                                                                 
 bidirectional_12 (Bidirecti  (None, None, 40)         13600     
 onal)                                                           
                                                                 
 bidirectional_13 (Bidirecti  (None, 40)               9760      
 onal)                                                           
                                                                 
 dense_5 (Dense)             (None, 1)                 41        
                                                                 
Total params: 663,401
Trainable params: 663,401
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import numpy as np # linear algebra

model.fit(np.array(train_x), np.array(train_y), epochs = 3, verbose = 1, 
          validation_data = (np.array(val_x), np.array(val_y)))

Epoch 1/3
352/352 [==============================] - 103s 273ms/step - loss: 0.4353 - accuracy: 0.7956 - val_loss: 0.4280 - val_accuracy: 0.8107
Epoch 2/3
352/352 [==============================] - 93s 265ms/step - loss: 0.2183 - accuracy: 0.9237 - val_loss: 0.2829 - val_accuracy: 0.8976
Epoch 3/3
352/352 [==============================] - 95s 268ms/step - loss: 0.1326 - accuracy: 0.9584 - val_loss: 0.2923 - val_accuracy: 0.8952


In [ ]:
print("Accuracy: ", model.evaluate(np.array(test_x), np.array(test_y))[1])

157/157 [==============================] - 8s 50ms/step - loss: 0.3094 - accuracy: 0.8934
Accuracy:  0.8934000134468079
